In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP 上的端到端机器学习：MLOps 阶段 4：评估：开始使用 AutoML 训练和 ML Metadata

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_vertex_ml_metadata_and_automl.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage4/get_started_with_vertex_ml_metadata_and_automl.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage4/get_started_with_vertex_ml_metadata_and_automl.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示如何在 Google Cloud 上使用 Vertex AI 进行端到端 MLOps 生产。本教程涵盖阶段 4：评估：使用 Vertex ML Metadata 和 AutoML 入门。

### 目标

在本教程中，您将学习如何使用`AutoML`进行训练，并为`Vertex ML Metadata`组装相应的工件链接。

此教程使用以下谷歌云ML服务：

- `AutoML 训练`
- `Vertex ML Metadata`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 创建一个 `Dataset` 资源。
- 创建一个对应的 `google.VertexDataset` 工件。
- 使用`AutoML`训练模型。
- 创建一个对应的 `google.VertexModel` 工件。
- 创建一个 `Endpoint` 资源。
- 创建一个对应的 `google.Endpoint` 工件。
- 将训练模型部署到`Endpoint`。
- 为`AutoML`训练作业和部署创建一个执行和上下文。
- 将对应的工件和上下文添加到执行中。
- 向执行添加工件链接（事件）。
- 显示执行图。

### 数据集

本教程使用的数据集为[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。本教程中的数据集版本存储在公共云存储桶中。训练模型可以预测给定图像中的花卉类型，包括五种花：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 成本
该教程使用了谷歌云的收费组件：

- Vertex AI
- 云存储

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing) 的信息，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预期使用量生成成本估算。 

## 安装

安装执行此笔记本所需的包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-storage $USER_FLAG -q

重新启动内核

安装完附加包之后，你需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###设置您的Google Cloud项目

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得300美元的免费信用额度，可用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI、Compute Engine和Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter将带有`!`前缀的行视为shell命令，并将前缀为`$`的Python变量插入这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

获取你的项目编号

现在项目ID已设置，您将获得与之对应的项目编号。

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

#### 地区

您还可以更改 `REGION` 变量，该变量用于笔记本的其余操作。以下是 Vertex AI 支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不可以使用多地域存储桶与 Vertex AI 进行训练。并非所有地区都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加直播教程会话，您可能会使用共享测试账户或项目。为避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 账户

**如果您正在使用 Vertex AI Workbench 记事本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用 Colab**，请运行下面的单元格，并按提示进行身份验证，通过 oAuth。

**否则**，请按照以下步骤操作：

在 Cloud Console 中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **单击创建服务帐号**。

2. 在 **服务帐号名称** 字段中输入名称，然后单击 **创建**。

3. 在 **授予此服务帐号对项目的访问权限** 部分，单击角色下拉列表。在过滤框中输入 "Vertex AI"，并选择 **Vertex AI 管理员**。在过滤框中输入 "Storage Object Admin"，并选择 **存储对象管理员**。

4. 单击创建。包含您密钥的 JSON 文件将下载到本地环境。

5. 在下面的单元格中将您的服务帐号密钥路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境是什么，下面的步骤都是必需的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储的暂存桶。这个暂存桶是您的数据集和模型资源在不同会话中保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括您所在组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查存储桶的内容验证对云存储的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import google.cloud.aiplatform_v1beta1 as aip_beta

初始化用于Python的Vertex AI SDK，为您的项目和相应的存储桶。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for `ML Metadata` services.

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## 设置客户端

Vertex 作为一个客户端/服务器模型运行。在您的一侧（Python脚本）上，您将创建一个客户端，该客户端向Vertex AI服务器发送请求并接收响应。

在本教程中，您将为工作流中的不同步骤使用不同的客户端。因此，请提前设置它们。

- Metadata服务用于创建记录、搜索和分析工件和元数据。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_metadata_client():
    client = aip_beta.MetadataServiceClient(client_options=client_options)
    return client


clients = {}
clients["metadata"] = create_metadata_client()

for client in clients.items():
    print(client)

自动机器学习训练任务

AutoML可以用来自动训练各种类型的图像模型。 AutoML自动化了以下内容：

- 数据集预处理
- 特征工程
- 数据馈送
- 模型架构选择
- 超参数调整
- 训练模型

了解更多关于[AutoML用户的Vertex AI](https://cloud.google.com/vertex-ai/docs/start/automl-users)。

## 自动机器学习图像模型

AutoML 可以训练以下类型的图像模型：

- 分类
- 目标检测
- 分割

模型可以训练用于部署到云端或导出到边缘设备。

了解更多关于[AutoML 模型类型](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)。

云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

本教程使用存储在公共云存储桶中的Happy Moments数据集的版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行数据。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式：
  - `single_label`：二元和多类分类
  - `multi_label`：多标签多类分类
  - `bounding_box`：目标检测
  - `image_segmentation`：分割

了解更多关于[ImageDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 为 `Vertex AI Dataset` 创建一个 `Artifact` 资源

接下来，您可以使用 `create_artifact()` 方法创建一个 `Artifact` 资源，具有以下参数：

- `parent`：指向 `Metadatastore` 资源的完全限定资源标识符 -- 在本示例中，您使用默认的 Metadatastore。
- `artifact`：`Artifact` 资源的定义
    - `display_name`：`Artifact` 资源的可读名称。
    - `uri`：artifact 文件的统一资源标识符。在本示例中，使用 `Dataset` 资源标识符，加上 `https://{REGION}-aiplatform.googleapis.com/v1/`
    - `name`：artifact 文件的资源名称。在本示例中，使用 `Dataset` 资源标识符。
    - `labels`：要分配给 `Artifact` 资源的用户自定义标签。
    - `schema_title`：描述元数据的模式的标题。在本示例中为：`google.VertexDataset`。
    - `metadata`：要与 `Artifact` 资源关联的元数据键值对。
- `artifact_id`：（可选）`Artifact` 资源的用户定义的短 ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Artifact

artifact_item = Artifact(
    display_name="flowers_" + TIMESTAMP,
    uri=f"https://{REGION}-aiplatform.googleapis.com/v1/" + dataset.resource_name,
    name=dataset.resource_name,
    schema_title="google.VertexDataset",
    metadata={"data_type": "image", "annotation_type": "image classification"},
    labels={"my_label": "value"},
)

artifact_dataset = clients["metadata"].create_artifact(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    artifact=artifact_item,
    artifact_id=dataset.resource_name.split("/")[-1],
)

print(artifact_dataset)

### 创建和运行训练流水线

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练流水线，2）运行这个流水线。

#### 创建训练流水线

使用`AutoMLImageTrainingJob`类创建一个AutoML训练流水线，参数如下：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签(`False`)或多标签(`True`)。
- `model_type`：部署的模型类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：优化精度而非延迟的部署于Google Cloud。
  - `CLOUD_LOW_LATENCY_`：优化延迟而非精度的部署于Google Cloud。
  - `MOBILE_TF_VERSATILE_1`：部署于边缘设备。
  - `MOBILE_TF_HIGH_ACCURACY_1`：优化精度而非延迟的部署于边缘设备。
  - `MOBILE_TF_LOW_LATENCY_1`：优化延迟而非精度的部署于边缘设备。
- `base_model`：(可选) 从现有`Model`资源进行迁移学习，仅支持图像分类。

实例化的对象是训练作业的有向无环图(DAG)。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

运行训练流程

接下来，您可以运行创建的DAG来启动训练作业，通过调用方法`run`，并传入以下参数：

- `dataset`: 用于训练模型的`Dataset`资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`: （可选）以毫节点小时为单位指定的最大训练时间（1000 = 节点小时）。
- `disable_early_stopping`: 如果为`True`，当服务认为无法进一步改进模型目标度量时，训练可能会在使用完整预算之前完成。

完成`run`方法后，将返回`Model`资源。

训练流程的执行可能需要超过30分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

### 为 `Vertex AI Model` 创建一个 `Artifact` 资源

接下来，您可以使用 `create_artifact()` 方法为 `Artifact` 资源创建一个资源，其中包括以下参数：

- `parent`: `Metadatastore` 资源的完全限定资源标识符，本示例中使用默认的 Metadatastore。
- `artifact`: `Artifact` 资源的定义
    - `display_name`: `Artifact` 资源的可读名称。
    - `uri`: artifact 文件的统一资源标识符。在这个示例中，使用 `Model` 资源标识符，前面加上 `https://{REGION}-aiplatform.googleapis.com/v1/`
    - `name`: artifact 文件的资源名称。在这个示例中，使用 `Model` 资源标识符。
    - `labels`: 用户定义的要分配给 `Artifact` 资源的标签。
    - `schema_title`: 描述元数据的模式的标题。在这个示例中是: `google.VertexModel`。
    - `metadata`: 与 `Artifact` 资源关联的元数据键值对。
- `artifact_id`: (可选) 用户定义的 `Artifact` 资源的短 ID。

In [ ]:
artifact_item = Artifact(
    display_name="flowers_" + TIMESTAMP,
    uri=f"https://{REGION}-aiplatform.googleapis.com/v1/" + model.resource_name,
    name=model.resource_name,
    schema_title="google.VertexModel",
    metadata={"model_type": "image classification"},
    labels={"my_label": "value"},
)

artifact_model = clients["metadata"].create_artifact(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    artifact=artifact_item,
    artifact_id=model.resource_name.split("/")[-1],
)

print(artifact_model)

## 查看模型评估分数

在模型训练完成之后，您可以使用`list_model_evaluations()`方法来查看模型的评估分数。该方法将返回每个评估切片的迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

### 为`Vertex AI EvaluationMetrics`创建一个`Artifact`资源

接下来，您可以使用`create_artifact()`方法为`Artifact`资源创建一个资源，具有以下参数：

- `parent`：指向`Metadatastore`资源的完全限定资源标识符--在此示例中，您可以使用默认的Metadatastore。
- `artifact`：`Artifact`资源的定义
    - `display_name`：`Artifact`资源的可读名称。
    - `uri`：工件文件的统一资源标识符。在此示例中，使用`https://{REGION}-aiplatform.googleapis.com/v1/`前缀的`EvaluationMetrics`资源标识符。
    - `name`：工件文件的资源名称。在此示例中，使用`EvaluationMetrics`资源标识符。
    - `labels`：分配给`Artifact`资源的用户定义标签。
    - `schema_title`：描述元数据的架构的标题。在此示例中：`system.SlicedClassificationMetrics`。
    - `metadata`：要与`Artifact`资源关联的元数据键/值对。
- `artifact_id`：（可选）为`Artifact`资源定义的用户定义短ID。

In [ ]:
artifact_item = Artifact(
    display_name="flowers_" + TIMESTAMP,
    uri=f"https://{REGION}-aiplatform.googleapis.com/v1/"
    + model_evaluations[0].resource_name,
    name=model_evaluations[0].resource_name,
    schema_title="system.SlicedClassificationMetrics",
    metadata={"param": "value"},
    labels={"my_label": "value"},
)

artifact_metrics = clients["metadata"].create_artifact(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    artifact=artifact_item,
    artifact_id=model_evaluations[0].resource_name.split("/")[-1],
)

print(artifact_metrics)

部署模型

接下来，部署您的模型以进行在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

为`Vertex AI Endpoint`创建一个`Artifact`资源

接下来，您可以使用`create_artifact()`方法创建一个`Artifact`资源，使用以下参数：

- `parent`：指向`Metadatastore`资源的完全限定资源标识符--在本例中，您使用默认的Metadatastore。
- `artifact`：`Artifact`资源的定义
    - `display_name`：`Artifact`资源的可读名称。
    - `uri`：工件文件的统一资源标识符。在本例中，`Endpoint`资源标识符前面加上`https://{REGION}-aiplatform.googleapis.com/v1/`
    - `name`：工件文件的资源名称。在本例中，`Endpoint`资源标识符。
    - `labels`：要分配给`Artifact`资源的用户定义标签。
    - `schema_title`：描述元数据的模式的标题。在本例中：`google.VertexEndpoint`。
    - `metadata`：要与`Artifact`资源关联的元数据键/值对。
- `artifact_id`：（可选）为`Artifact`资源定义的用户定义短ID。

In [ ]:
artifact_item = Artifact(
    display_name="flowers_" + TIMESTAMP,
    uri=f"https://{REGION}-aiplatform.googleapis.com/v1/" + endpoint.resource_name,
    name=endpoint.resource_name,
    schema_title="google.VertexEndpoint",
    metadata={"param": "value"},
    labels={"my_label": "value"},
)

artifact_endpoint = clients["metadata"].create_artifact(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    artifact=artifact_item,
    artifact_id=endpoint.resource_name.split("/")[-1],
)

print(artifact_endpoint)

### 创建一个`Execution`资源

一系列带有输入和输出的步骤被称为一个执行。

接下来，您可以使用`create_execution()`方法为AutoML训练和模型部署创建一个`Execution`资源，具有以下参数：

- `parent`：（可选）指向`Metadatastore`资源的完全限定资源标识符。
- `execution`：
    - `display_name`：一个易于阅读的`Execution`资源的名称。
    - `schema_title`：描述元数据的模式的标题。在这个例子中：`system.ContainerExecution`。
    - `metadata`：要与`Execution`资源关联的元数据键值对。
- `execution_id`：（可选）用户定义的`Execution`资源的短ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Execution

execution = clients["metadata"].create_execution(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    execution=Execution(
        display_name="AutoML training and deployment",
        schema_title="system.ContainerExecution",
        metadata={"model_type": "image classification"},
    ),
    execution_id=dag.resource_name.split("/")[-1],
)

print(execution)

### 将事件添加到`Execution`资源

`Execution`资源由执行过程中发生的事件序列组成。每个事件由一个作为`Execution`资源的输入或输出的工件组成。

您可以使用`add_execution_events()`方法将执行事件添加到`Execution`资源中，具有以下参数：

- `execution`：`Execution`资源的完全限定资源标识符。在本示例中，执行AutoML训练和部署的步骤。
- `events`：构成执行过程的事件序列。在本示例中，工件包括：VertexDataset、VertexModel、SlicedClassificationMetrics和VertexEndpoint。
    - `artifact_dataset`：执行的输入。
    - `artifact_model`：
    - `artifact_metrics`：
    - `artifact_endpoint`：执行的输出。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Event

clients["metadata"].add_execution_events(
    execution=execution.name,
    events=[
        Event(
            artifact=artifact_dataset.name,
            type_=Event.Type.INPUT,
            labels={"my_label": "my_value"},
        ),
        Event(
            artifact=artifact_model.name,
            type_=Event.Type.OUTPUT,
            labels={"my_label": "my_value"},
        ),
        Event(
            artifact=artifact_metrics.name,
            type_=Event.Type.OUTPUT,
            labels={"my_label": "my_value"},
        ),
        Event(
            artifact=artifact_endpoint.name,
            type_=Event.Type.OUTPUT,
            labels={"my_label": "my_value"},
        ),
    ],
)

### 创建`Context`资源

`Context`资源表示由一个或多个执行组成的上下文。您可以使用`create_context()`方法创建一个`Context`资源，并提供以下参数：

- `parent`：指向`Metadatastore`资源的完全限定资源标识符。
- `context`：
    - `display_name`：`Execution`资源的人类可读名称。
    - `schema_title`：描述元数据的架构标题。在此示例中为：`system.Pipeline`。
    - `labels`：用户定义的标签，用于分配给`Context`资源。
    - `metadata`：要与`Execution`资源关联的元数据键值对。
- `context_id`：（可选）用户定义的用于`Context`资源的短ID。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import Context

context = clients["metadata"].create_context(
    parent=f"projects/{PROJECT_NUMBER}/locations/{REGION}/metadataStores/default",
    context=Context(
        display_name="flowers_" + TIMESTAMP,
        schema_title="system.Pipeline",
        metadata={"param": "value"},
        labels=[{"my_label", "my_value"}],
    ),
    context_id=dag.resource_name.split("/")[-1],
)

print(context)

### 将工件和执行对象合并为上下文

上下文用于将`Artifact`资源和`Execution`资源组合在一个可查询和类型化的类别下。上下文可用于表示元数据集合。

使用`add_context_artifacts_and_executions()`方法将一组`Artifact`和`Execution`资源合并为一个`Context`资源，参数如下：

- `context`：`Context`资源的完全限定资源标识符。
- `artifacts`：`Artifact`资源的完全限定资源标识符的列表。
- `executions`：`Execution`资源的完全限定资源标识符的列表。

In [ ]:
clients["metadata"].add_context_artifacts_and_executions(
    context=context.name,
    artifacts=[
        artifact_dataset.name,
        artifact_model.name,
        artifact_metrics.name,
        artifact_endpoint.name,
    ],
    executions=[execution.name],
)

### 查询上下文

您可以使用方法`query_context_lineage_subgraph()`来查询`Context`资源的子图，具有以下参数：

- `context`：`Context`资源的完全限定资源标识符。

In [ ]:
subgraph = clients["metadata"].query_context_lineage_subgraph(context=context.name)

print(subgraph)

在界面上查看图表

最后，进入“Vertex AI ML Metadata”界面，并选择任何一个你创建的工件。它会类似于：

<img src='automl_mlmd.png'/>

### 删除一个`Execution`资源

您可以使用`delete_execution()`方法删除一个`Execution`资源，参数如下：

- `name`：`Execution`资源的完全限定资源标识符。

In [ ]:
clients["metadata"].delete_execution(name=execution.name)

### 删除 `Context` 资源

您可以使用 `delete_context()` 方法删除 `Context` 资源，参数如下：

- `name`：`Context` 资源的完全限定资源标识符。

In [ ]:
clients["metadata"].delete_context(name=context.name)

### 删除一个`Artifact`资源

您可以使用`delete_artifact()`方法删除一个`Artifact`资源，需要以下参数：

- `name`：`Artifact`资源的完全限定资源标识符。

In [ ]:
clients["metadata"].delete_artifact(name=artifact_dataset.name)
clients["metadata"].delete_artifact(name=artifact_model.name)
clients["metadata"].delete_artifact(name=artifact_metrics.name)
clients["metadata"].delete_artifact(name=artifact_endpoint.name)

取消部署模型

当您完成预测时，您可以从“端点”资源中取消部署模型。这将取消所有计算资源，并结束已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

删除模型

方法 'delete()' 将删除模型。

In [ ]:
model.delete()

删除数据集

方法'delete()'将删除数据集。

In [ ]:
dataset.delete()

#### 删除该端点

方法 'delete()' 将会删除该端点。

In [ ]:
endpoint.delete()

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI